In [1]:
import numpy as np
import pickle
import os

seed = 2023

In [2]:
import torch

# set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [3]:
from nlpsig_networks.scripts.seqsignet_functions import seqsignet_hyperparameter_search

In [4]:
output_dir = "client_talk_type_output"
if not os.path.isdir(output_dir):
    os.makedirs(output_dir)

## AnnoMI

In [5]:
%run ../load_anno_mi.py

In [6]:
anno_mi.head()

,mi_quality,transcript_id,topic,utterance_id,interlocutor,timestamp,utterance_text,annotator_id,therapist_input_exists,therapist_input_subtype,reflection_exists,reflection_subtype,question_exists,question_subtype,main_therapist_behaviour,client_talk_type,datetime,speaker
0,high,0,reducing alcohol consumption,0,therapist,00:00:13,Thanks for filling it out. We give this form t...,3,False,NaN,False,NaN,True,open,question,NaN,2023-11-02 00:00:13,-1
1,high,0,reducing alcohol consumption,1,client,00:00:24,Sure.,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,neutral,2023-11-02 00:00:24,1
2,high,0,reducing alcohol consumption,2,therapist,00:00:25,"So, let's see. It looks that you put-- You dri...",3,True,information,False,NaN,False,NaN,therapist_input,NaN,2023-11-02 00:00:25,-1
3,high,0,reducing alcohol consumption,3,client,00:00:34,Mm-hmm.,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,neutral,2023-11-02 00:00:34,1
4,high,0,reducing alcohol consumption,4,therapist,00:00:34,-and you usually have three to four drinks whe...,3,True,information,False,NaN,False,NaN,therapist_input,NaN,2023-11-02 00:00:34,-1


In [7]:
with open("../anno_mi_sbert.pkl", "rb") as f:
    sbert_embeddings = pickle.load(f)

sbert_embeddings.shape

(9699, 384)

In [8]:
features = ["time_encoding", "timeline_index"]
standardise_method = ["z_score", None]
include_features_in_path = True
include_features_in_input = True

In [9]:
num_epochs = 100
dimensions = [15]
swnu_hidden_dim_sizes_and_sig_depths = [([12], 3), ([10], 4)]
lstm_hidden_dim_sizes = [384]
ffn_hidden_dim_sizes = [[256, 256], [512, 512]]
dropout_rates = [0.1, 0.2]
learning_rates = [1e-3, 1e-4, 5e-4]
seeds = [1, 12, 123]
loss = "focal"
gamma = 2
validation_metric = "f1"
patience = 5

In [10]:
kwargs = {
    "num_epochs": num_epochs,
    "df": anno_mi,
    "id_column": "transcript_id",
    "label_column": "client_talk_type",
    "embeddings": sbert_embeddings,
    "y_data": y_data_client,
    "output_dim": output_dim_client,
    "dimensions": dimensions,
    "log_signature": True,
    "pooling": "signature",
    "swnu_hidden_dim_sizes_and_sig_depths": swnu_hidden_dim_sizes_and_sig_depths,
    "lstm_hidden_dim_sizes": lstm_hidden_dim_sizes,
    "ffn_hidden_dim_sizes": ffn_hidden_dim_sizes,
    "dropout_rates": dropout_rates,
    "learning_rates": learning_rates,
    "BiLSTM": True,
    "seeds": seeds,
    "loss": loss,
    "gamma": gamma,
    "device": device,
    "features": features,
    "standardise_method": standardise_method,
    "include_features_in_path": include_features_in_path,
    "include_features_in_input": include_features_in_input,
    "path_indices": client_index,
    "split_ids": client_transcript_id,
    "k_fold": True,
    "patience": patience,
    "validation_metric": validation_metric,
    "verbose": False,
}

# history_length=11

In [11]:
shift = 3
window_size = 5
n = 3

## GRP

In [12]:
(
    seqsignet_network_grp_kfold_11,
    best_seqsignet_network_grp_kfold_11,
    _,
    __,
) = seqsignet_hyperparameter_search(
    shift=shift,
    window_size=window_size,
    n=n,
    dim_reduce_methods=["gaussian_random_projection"],
    results_output=f"{output_dir}/seqsignet_grp_focal_{gamma}_{shift}_{window_size}_{n}_kfold.csv",
    **kwargs,
)

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]


##################################################
dimension: 15 | method: gaussian_random_projection
given shift 3, window size 5 and n 3: history length = 11
[INFO] Concatenating the embeddings to the dataframe...
[INFO] - columns beginning with 'e' denote the full embddings.
[INFO] - columns beginning with 'd' denote the dimension reduced embeddings.
[INFO] Adding time feature columns into dataframe in `.df`.
[INFO] Adding 'time_encoding' feature...
[INFO] Adding 'time_diff' feature...
[INFO] Adding 'timeline_index' feature...
[INFO] Padding ids and storing in `.df_padded` and `.array_padded` attributes.


  0%|          | 0/9699 [00:00<?, ?it/s]

[INFO] The path was created for each item in the dataframe, by looking at its history, so to include embeddings in the FFN input, we concatenate the embeddings for each sentence / text.


...

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

given shift 3, window size 5 and n 3: history length = 11
[INFO] Concatenating the embeddings to the dataframe...
[INFO] - columns beginning with 'e' denote the full embddings.
[INFO] - columns beginning with 'd' denote the dimension reduced embeddings.
[INFO] Adding time feature columns into dataframe in `.df`.
[INFO] Adding 'time_encoding' feature...
[INFO] Adding 'time_diff' feature...
[INFO] Adding 'timeline_index' feature...
[INFO] Padding ids and storing in `.df_padded` and `.array_padded` attributes.


  0%|          | 0/9699 [00:00<?, ?it/s]

[INFO] The path was created for each item in the dataframe, by looking at its history, so to include embeddings in the FFN input, we concatenate the embeddings for each sentence / text.
saving results dataframe to CSV for this hyperparameter search in client_talk_type_output/seqsignet_grp_focal_2_3_5_3_kfold.csv
saving the best model results dataframe to CSV for this hyperparameter search in client_talk_type_output/seqsignet_grp_focal_2_3_5_3_kfold_best_model.csv


In [13]:
seqsignet_network_grp_kfold_11

,loss,accuracy,f1,f1_scores,precision,precision_scores,recall,recall_scores,valid_loss,valid_accuracy,...,BiLSTM,loss_function,gamma,k_fold,n_splits,augmentation_type,hidden_dim_aug,comb_method,batch_size,model_id
0,None,0.627778,0.492208,"[0.757208765859285, 0.39842726081258195, 0.320...",0.490554,"[0.7550316273720529, 0.4233983286908078, 0.293...",0.496727,"[0.7593984962406015, 0.37623762376237624, 0.35...",None,0.664740,...,True,focal,2,True,5,Conv1d,None,concatenation,64,0
0,None,0.621839,0.506363,"[0.744645799011532, 0.43302180685358255, 0.341...",0.499533,"[0.7722895309102206, 0.41002949852507375, 0.31...",0.516189,"[0.7189126662810873, 0.45874587458745875, 0.37...",None,0.658157,...,True,focal,2,True,5,Conv1d,None,concatenation,64,0
0,None,0.618966,0.501558,"[0.7406066093254866, 0.4389713871785585, 0.325...",0.502078,"[0.7743767750078889, 0.39122014202711425, 0.34...",0.506856,"[0.7096587622903412, 0.5, 0.3109090909090909]",None,0.657996,...,True,focal,2,True,5,Conv1d,None,concatenation,64,0
0,None,0.644061,0.519033,"[0.7743740795287188, 0.4280821917808219, 0.354...",0.512943,"[0.789015606242497, 0.44483985765124556, 0.304...",0.532148,"[0.760266049739734, 0.41254125412541254, 0.423...",None,0.665061,...,True,focal,2,True,5,Conv1d,None,concatenation,64,1
0,None,0.636398,0.512537,"[0.7630296766573158, 0.4324765401876785, 0.342...",0.506521,"[0.7794871794871795, 0.42776432606941084, 0.31...",0.520909,"[0.7472527472527473, 0.4372937293729373, 0.378...",None,0.671644,...,True,focal,2,True,5,Conv1d,None,concatenation,64,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,None,0.633142,0.496426,"[0.7675191446322784, 0.38869257950530034, 0.33...",0.493890,"[0.7669650591972278, 0.41825095057034223, 0.29...",0.503703,"[0.768074031231926, 0.36303630363036304, 0.38]",None,0.663777,...,True,focal,2,True,5,Conv1d,None,concatenation,64,22
0,None,0.631034,0.496051,"[0.7602759413624604, 0.38957710380179417, 0.33...",0.495392,"[0.7557142857142857, 0.4038972542072631, 0.326...",0.497347,"[0.764893001735107, 0.37623762376237624, 0.350...",None,0.672447,...,True,focal,2,True,5,Conv1d,None,concatenation,64,22
0,None,0.640230,0.525272,"[0.7610752365220004, 0.46522593320235756, 0.34...",0.516869,"[0.7916276163698844, 0.4441110277569392, 0.314...",0.537990,"[0.7327935222672065, 0.4884488448844885, 0.392...",None,0.654303,...,True,focal,2,True,5,Conv1d,None,concatenation,64,23
0,None,0.648467,0.510863,"[0.7789173789173789, 0.39284032030146016, 0.36...",0.512846,"[0.7675463222908478, 0.45773874862788144, 0.31...",0.520048,"[0.7906304222093696, 0.34405940594059403, 0.42...",None,0.666988,...,True,focal,2,True,5,Conv1d,None,concatenation,64,23


In [14]:
best_seqsignet_network_grp_kfold_11["f1"].mean()

0.5123820701801569

In [15]:
best_seqsignet_network_grp_kfold_11["precision"].mean()

0.5101369231220637

In [16]:
best_seqsignet_network_grp_kfold_11["recall"].mean()

0.5205559152406468

In [17]:
np.stack(best_seqsignet_network_grp_kfold_11["f1_scores"]).mean(axis=0)

array([0.76515619, 0.42276003, 0.34922999])

In [18]:
np.stack(best_seqsignet_network_grp_kfold_11["precision_scores"]).mean(axis=0)

array([0.76767588, 0.45053642, 0.31219847])

In [19]:
np.stack(best_seqsignet_network_grp_kfold_11["recall_scores"]).mean(axis=0)

array([0.76315789, 0.40154015, 0.3969697 ])

# history_length=20

In [20]:
shift = 3
window_size = 5
n = 6

## GRP

In [21]:
(
    seqsignet_network_grp_kfold_20,
    best_seqsignet_network_grp_kfold_20,
    _,
    __,
) = seqsignet_hyperparameter_search(
    shift=shift,
    window_size=window_size,
    n=n,
    dim_reduce_methods=["gaussian_random_projection"],
    results_output=f"{output_dir}/seqsignet_grp_focal_{gamma}_{shift}_{window_size}_{n}_kfold.csv",
    **kwargs,
)

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]


##################################################
dimension: 15 | method: gaussian_random_projection
given shift 3, window size 5 and n 6: history length = 20
[INFO] Concatenating the embeddings to the dataframe...
[INFO] - columns beginning with 'e' denote the full embddings.
[INFO] - columns beginning with 'd' denote the dimension reduced embeddings.
[INFO] Adding time feature columns into dataframe in `.df`.
[INFO] Adding 'time_encoding' feature...
[INFO] Adding 'time_diff' feature...
[INFO] Adding 'timeline_index' feature...
[INFO] Padding ids and storing in `.df_padded` and `.array_padded` attributes.


  0%|          | 0/9699 [00:00<?, ?it/s]

[INFO] The path was created for each item in the dataframe, by looking at its history, so to include embeddings in the FFN input, we concatenate the embeddings for each sentence / text.


...

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

given shift 3, window size 5 and n 6: history length = 20
[INFO] Concatenating the embeddings to the dataframe...
[INFO] - columns beginning with 'e' denote the full embddings.
[INFO] - columns beginning with 'd' denote the dimension reduced embeddings.
[INFO] Adding time feature columns into dataframe in `.df`.
[INFO] Adding 'time_encoding' feature...
[INFO] Adding 'time_diff' feature...
[INFO] Adding 'timeline_index' feature...
[INFO] Padding ids and storing in `.df_padded` and `.array_padded` attributes.


  0%|          | 0/9699 [00:00<?, ?it/s]

[INFO] The path was created for each item in the dataframe, by looking at its history, so to include embeddings in the FFN input, we concatenate the embeddings for each sentence / text.
saving results dataframe to CSV for this hyperparameter search in client_talk_type_output/seqsignet_grp_focal_2_3_5_6_kfold.csv
saving the best model results dataframe to CSV for this hyperparameter search in client_talk_type_output/seqsignet_grp_focal_2_3_5_6_kfold_best_model.csv


In [22]:
seqsignet_network_grp_kfold_20

,loss,accuracy,f1,f1_scores,precision,precision_scores,recall,recall_scores,valid_loss,valid_accuracy,...,BiLSTM,loss_function,gamma,k_fold,n_splits,augmentation_type,hidden_dim_aug,comb_method,batch_size,model_id
0,None,0.614176,0.499474,"[0.7403065825067628, 0.4195083267248216, 0.338...",0.491396,"[0.7706508135168961, 0.40381679389312974, 0.29...",0.512607,"[0.7122614227877386, 0.43646864686468645, 0.38...",None,0.656712,...,True,focal,2,True,5,Conv1d,None,concatenation,64,0
0,None,0.620306,0.500980,"[0.7498523331364442, 0.3909090909090909, 0.362...",0.493827,"[0.7661436330718165, 0.39155629139072845, 0.32...",0.511804,"[0.7342394447657605, 0.39026402640264024, 0.41...",None,0.660726,...,True,focal,2,True,5,Conv1d,None,concatenation,64,0
0,None,0.640230,0.512249,"[0.7625349213350978, 0.4493496255419787, 0.324...",0.510038,"[0.7756506132216572, 0.43018867924528303, 0.32...",0.515202,"[0.7498554077501446, 0.47029702970297027, 0.32...",None,0.668593,...,True,focal,2,True,5,Conv1d,None,concatenation,64,0
0,None,0.645785,0.521459,"[0.772306335440247, 0.43828715365239296, 0.353...",0.515466,"[0.7853512705530643, 0.4461538461538462, 0.314...",0.531339,"[0.7596876807403123, 0.4306930693069307, 0.403...",None,0.665703,...,True,focal,2,True,5,Conv1d,None,concatenation,64,1
0,None,0.643870,0.517084,"[0.7687114763721751, 0.43953394937725987, 0.34...",0.513646,"[0.7803933253873659, 0.4283476898981989, 0.332...",0.521080,"[0.7573742047426258, 0.4513201320132013, 0.354...",None,0.674213,...,True,focal,2,True,5,Conv1d,None,concatenation,64,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,None,0.645019,0.504332,"[0.7800518881522053, 0.4020933275185347, 0.330...",0.502511,"[0.7775862068965518, 0.42645698427382056, 0.30...",0.508844,"[0.7825332562174667, 0.3803630363036304, 0.363...",None,0.672929,...,True,focal,2,True,5,Conv1d,None,concatenation,64,22
0,None,0.641954,0.501703,"[0.7712287712287713, 0.4014084507042254, 0.332...",0.502469,"[0.7613412228796844, 0.43018867924528303, 0.31...",0.502841,"[0.7813765182186235, 0.37623762376237624, 0.35...",None,0.676140,...,True,focal,2,True,5,Conv1d,None,concatenation,64,22
0,None,0.637739,0.528412,"[0.757543593631539, 0.46435379336637445, 0.363...",0.519423,"[0.7963021995537137, 0.4316087880935507, 0.330...",0.542831,"[0.7223828802776171, 0.5024752475247525, 0.403...",None,0.649807,...,True,focal,2,True,5,Conv1d,None,concatenation,64,23
0,None,0.648467,0.518340,"[0.7726416461382408, 0.434928631402183, 0.3474...",0.516065,"[0.7743247168167295, 0.4427350427350427, 0.331...",0.521271,"[0.7709658762290341, 0.4273927392739274, 0.365...",None,0.669236,...,True,focal,2,True,5,Conv1d,None,concatenation,64,23


In [23]:
best_seqsignet_network_grp_kfold_20["f1"].mean()

0.5032739316970518

In [24]:
best_seqsignet_network_grp_kfold_20["precision"].mean()

0.4974416296564994

In [25]:
best_seqsignet_network_grp_kfold_20["recall"].mean()

0.5146013683689556

In [26]:
np.stack(best_seqsignet_network_grp_kfold_20["f1_scores"]).mean(axis=0)

array([0.75211517, 0.41411823, 0.34358839])

In [27]:
np.stack(best_seqsignet_network_grp_kfold_20["precision_scores"]).mean(axis=0)

array([0.7696442 , 0.41834931, 0.30433137])

In [28]:
np.stack(best_seqsignet_network_grp_kfold_20["recall_scores"]).mean(axis=0)

array([0.73616734, 0.41309131, 0.39454545])

# history_length=35

In [29]:
shift = 3
window_size = 5
n = 11

## GRP

In [30]:
(
    seqsignet_network_grp_kfold_35,
    best_seqsignet_network_grp_kfold_35,
    _,
    __,
) = seqsignet_hyperparameter_search(
    shift=shift,
    window_size=window_size,
    n=n,
    dim_reduce_methods=["gaussian_random_projection"],
    results_output=f"{output_dir}/seqsignet_grp_focal_{gamma}_{shift}_{window_size}_{n}_kfold.csv",
    **kwargs,
)

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]


##################################################
dimension: 15 | method: gaussian_random_projection
given shift 3, window size 5 and n 11: history length = 35
[INFO] Concatenating the embeddings to the dataframe...
[INFO] - columns beginning with 'e' denote the full embddings.
[INFO] - columns beginning with 'd' denote the dimension reduced embeddings.
[INFO] Adding time feature columns into dataframe in `.df`.
[INFO] Adding 'time_encoding' feature...
[INFO] Adding 'time_diff' feature...
[INFO] Adding 'timeline_index' feature...
[INFO] Padding ids and storing in `.df_padded` and `.array_padded` attributes.


  0%|          | 0/9699 [00:00<?, ?it/s]

[INFO] The path was created for each item in the dataframe, by looking at its history, so to include embeddings in the FFN input, we concatenate the embeddings for each sentence / text.


...

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

given shift 3, window size 5 and n 11: history length = 35
[INFO] Concatenating the embeddings to the dataframe...
[INFO] - columns beginning with 'e' denote the full embddings.
[INFO] - columns beginning with 'd' denote the dimension reduced embeddings.
[INFO] Adding time feature columns into dataframe in `.df`.
[INFO] Adding 'time_encoding' feature...
[INFO] Adding 'time_diff' feature...
[INFO] Adding 'timeline_index' feature...
[INFO] Padding ids and storing in `.df_padded` and `.array_padded` attributes.


  0%|          | 0/9699 [00:00<?, ?it/s]

[INFO] The path was created for each item in the dataframe, by looking at its history, so to include embeddings in the FFN input, we concatenate the embeddings for each sentence / text.
saving results dataframe to CSV for this hyperparameter search in client_talk_type_output/seqsignet_grp_focal_2_3_5_11_kfold.csv
saving the best model results dataframe to CSV for this hyperparameter search in client_talk_type_output/seqsignet_grp_focal_2_3_5_11_kfold_best_model.csv


In [31]:
seqsignet_network_grp_kfold_35

,loss,accuracy,f1,f1_scores,precision,precision_scores,recall,recall_scores,valid_loss,valid_accuracy,...,BiLSTM,loss_function,gamma,k_fold,n_splits,augmentation_type,hidden_dim_aug,comb_method,batch_size,model_id
0,None,0.645594,0.517672,"[0.767187271931105, 0.4480519480519481, 0.3377...",0.515288,"[0.7745358090185677, 0.44089456869009586, 0.33...",0.520292,"[0.7599768652400232, 0.45544554455445546, 0.34...",None,0.668433,...,True,focal,2,True,5,Conv1d,None,concatenation,64,0
0,None,0.628927,0.513232,"[0.7494308696312035, 0.4638755094479437, 0.326...",0.507280,"[0.7885659533695305, 0.4209818426361802, 0.312...",0.524105,"[0.7139965297860035, 0.5165016501650165, 0.341...",None,0.662010,...,True,focal,2,True,5,Conv1d,None,concatenation,64,0
0,None,0.653831,0.515296,"[0.7756956274843839, 0.4305010893246187, 0.339...",0.519125,"[0.7618516452872282, 0.45614035087719296, 0.33...",0.512548,"[0.790052053209948, 0.4075907590759076, 0.34]",None,0.668915,...,True,focal,2,True,5,Conv1d,None,concatenation,64,0
0,None,0.651724,0.521622,"[0.774850822296609, 0.4524207011686144, 0.3375...",0.518515,"[0.7799589803691767, 0.4577702702702703, 0.317...",0.525668,"[0.7698091382301908, 0.4471947194719472, 0.36]",None,0.670841,...,True,focal,2,True,5,Conv1d,None,concatenation,64,1
0,None,0.633908,0.509552,"[0.7614705882352941, 0.41885245901639345, 0.34...",0.504116,"[0.774685816876122, 0.4161237785016287, 0.3215...",0.516772,"[0.7486986697513013, 0.42161716171617164, 0.38]",None,0.671002,...,True,focal,2,True,5,Conv1d,None,concatenation,64,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,None,0.635632,0.496297,"[0.7721683784174742, 0.3899261832392531, 0.326...",0.493596,"[0.7725036179450072, 0.41154903758020167, 0.29...",0.501977,"[0.7718334297281666, 0.3704620462046205, 0.363...",None,0.666185,...,True,focal,2,True,5,Conv1d,None,concatenation,64,22
0,None,0.645211,0.514415,"[0.7684700407213496, 0.4414414414414415, 0.333...",0.512700,"[0.7729666471620831, 0.43821138211382116, 0.32...",0.516248,"[0.7640254482359745, 0.44471947194719474, 0.34]",None,0.671805,...,True,focal,2,True,5,Conv1d,None,concatenation,64,22
0,None,0.607088,0.497649,"[0.7371290127195639, 0.3964329144710174, 0.359...",0.487896,"[0.7736808645899556, 0.3896414342629482, 0.300...",0.518204,"[0.7038750722961249, 0.4034653465346535, 0.447...",None,0.653661,...,True,focal,2,True,5,Conv1d,None,concatenation,64,23
0,None,0.630077,0.512236,"[0.7537479590322101, 0.42942345924453285, 0.35...",0.505968,"[0.7743214394632509, 0.41442824251726784, 0.32...",0.520534,"[0.7342394447657605, 0.44554455445544555, 0.38...",None,0.665543,...,True,focal,2,True,5,Conv1d,None,concatenation,64,23


best_seqsignet_network_grp_kfold_35["f1"].mean()

In [32]:
best_seqsignet_network_grp_kfold_35["precision"].mean()

0.5067860229271276

In [33]:
best_seqsignet_network_grp_kfold_35["recall"].mean()

0.5253365384735891

In [34]:
np.stack(best_seqsignet_network_grp_kfold_35["f1_scores"]).mean(axis=0)

array([0.75975717, 0.42584103, 0.35191066])

In [35]:
np.stack(best_seqsignet_network_grp_kfold_35["precision_scores"]).mean(axis=0)

array([0.77404948, 0.44261133, 0.30369726])

In [36]:
np.stack(best_seqsignet_network_grp_kfold_35["recall_scores"]).mean(axis=0)

array([0.74599961, 0.41061606, 0.41939394])

# history_length=80

In [37]:
shift = 3
window_size = 5
n = 26

## GRP

In [ ]:
(
    seqsignet_network_grp_kfold_80,
    best_seqsignet_network_grp_kfold_80,
    _,
    __,
) = seqsignet_hyperparameter_search(
    shift=shift,
    window_size=window_size,
    n=n,
    dim_reduce_methods=["gaussian_random_projection"],
    results_output=f"{output_dir}/seqsignet_grp_focal_{gamma}_{shift}_{window_size}_{n}_kfold.csv",
    **kwargs,
)

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]


##################################################
dimension: 15 | method: gaussian_random_projection
given shift 3, window size 5 and n 26: history length = 80
[INFO] Concatenating the embeddings to the dataframe...
[INFO] - columns beginning with 'e' denote the full embddings.
[INFO] - columns beginning with 'd' denote the dimension reduced embeddings.
[INFO] Adding time feature columns into dataframe in `.df`.
[INFO] Adding 'time_encoding' feature...
[INFO] Adding 'time_diff' feature...
[INFO] Adding 'timeline_index' feature...
[INFO] Padding ids and storing in `.df_padded` and `.array_padded` attributes.


  0%|          | 0/9699 [00:00<?, ?it/s]

[INFO] The path was created for each item in the dataframe, by looking at its history, so to include embeddings in the FFN input, we concatenate the embeddings for each sentence / text.


...

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

In [ ]:
seqsignet_network_grp_kfold_80

In [ ]:
best_seqsignet_network_grp_kfold_80["f1"].mean()

In [ ]:
best_seqsignet_network_grp_kfold_80["precision"].mean()

In [ ]:
best_seqsignet_network_grp_kfold_80["recall"].mean()

In [ ]:
np.stack(best_seqsignet_network_grp_kfold_80["f1_scores"]).mean(axis=0)

In [ ]:
np.stack(best_seqsignet_network_grp_kfold_80["precision_scores"]).mean(axis=0)

In [ ]:
np.stack(best_seqsignet_network_grp_kfold_80["recall_scores"]).mean(axis=0)

# history_length=110

In [ ]:
shift = 3
window_size = 5
n = 36

## GRP

In [ ]:
(
    seqsignet_network_grp_kfold_110,
    best_seqsignet_network_grp_kfold_110,
    _,
    __,
) = seqsignet_hyperparameter_search(
    shift=shift,
    window_size=window_size,
    n=n,
    dim_reduce_methods=["gaussian_random_projection"],
    results_output=f"{output_dir}/seqsignet_grp_focal_{gamma}_{shift}_{window_size}_{n}_kfold.csv",
    **kwargs,
)

In [ ]:
seqsignet_network_grp_kfold_110

In [ ]:
best_seqsignet_network_grp_kfold_110["f1"].mean()

In [ ]:
best_seqsignet_network_grp_kfold_110["precision"].mean()

In [ ]:
best_seqsignet_network_grp_kfold_110["recall"].mean()

In [ ]:
np.stack(best_seqsignet_network_grp_kfold_110["f1_scores"]).mean(axis=0)

In [ ]:
np.stack(best_seqsignet_network_grp_kfold_110["precision_scores"]).mean(axis=0)

In [ ]:
np.stack(best_seqsignet_network_grp_kfold_110["recall_scores"]).mean(axis=0)